In [1]:
import os
import pandas as pd
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np


In [8]:
import ultralytics
ultralytics.checks()   

Ultralytics YOLOv8.2.2 🚀 Python-3.8.19 torch-2.3.0+cu121 CUDA:0 (Tesla V100S-PCIE-32GB, 32494MiB)
Setup complete ✅ (15 CPUs, 43.0 GB RAM, 268.9/290.6 GB disk)


In [2]:
data = pd.read_csv('../data/train_labels_yolo.csv')
txt_dir = '../data/labels/'

os.makedirs(txt_dir, exist_ok=True)

# group data by image_id and write to individual files
for image_id, group in data.groupby('image_id'):
    # construct file path
    file_path = os.path.join(txt_dir, f"{image_id}.txt")
    with open(file_path, 'w') as file:
        for _, row in group.iterrows():
            # write the class ID (0) and the bounding box coordinates to the file
            file.write(f"0 {row['x_center']} {row['y_center']} {row['width']} {row['height']}\n")


In [19]:
def count_files(folder_path):
    files = os.listdir(folder_path)
    file_count = len([file for file in files if os.path.isfile(os.path.join(folder_path, file))])
    return file_count

label_path = '../data/labels/train/'
image_path = '../data/images/train'

print("Number of files in the label:", count_files(label_path))
print("Number of files in the image:", count_files(image_path))


Number of files in the label: 22894
Number of files in the image: 22894


In [21]:
image_dir = '../data/images/train'
label_dir = '../data/labels/train'

image_files = {os.path.splitext(file)[0] for file in os.listdir(image_dir) if file.endswith('.png')}
label_files = {os.path.splitext(file)[0] for file in os.listdir(label_dir) if file.endswith('.txt')}

# print(image_files)
images_without_labels = image_files - label_files
labels_without_images = label_files - image_files

if images_without_labels:
    print(f"Images without labels: {len(images_without_labels)}")
    for image in images_without_labels:
        print(image)
else:
    print("All images have corresponding labels.")

if labels_without_images:
    print(f"Labels without images: {len(labels_without_images)}")
    for label in labels_without_images:
        print(label)
else:
    print("All labels have corresponding images.")


{'cb8ca4d78927cce2ef3bb2e1f4b70759', 'd3ad6e735690bf3d84a9b9b8b5378161', 'e5100198d028f25442faf3f3e3cac278', '528e909e6d93ecbfb27aca7d57f5c153', '32d849be41856cea483759ccf7259615', 'f972f9b42574e436a2a9e10c90508a52', '1425414c71cbc155a85b137a7faa75c4', 'c3ea712e719abe3753356e28bde01967', '6b092c1ac1604a8a6800479497a081b8', 'c9fdd755dbc4d305a86236350930694f', 'ef73ee5d550b328b623a26f98c660723', 'b8f6e425a176382f062a4f840f262374', '50a44d42b10929db6cbc4f353a5e0eba', '2e34d15503d52d1716621f1829c90302', 'e22e1a7418701556a4f576df3650743e', '4a8d1bfecf0eb78b09153cf91e26eb2d', '7107a251f2817d4b7fea735f27fef6ec', '1c9dc89c00310454dade0f387ca8583c', '135018239142849ae1b9efbc5f0a1a7e', 'ae4f4937e968413b5d8ba1f7b6d5d87a', 'ff03f4a268102f71819d535760264518', 'edf0f9cbfca21069d3f99866db9cc188', 'de2af7e2964b58f4c38e53117fbe92dd', 'a9cdc87f8289f81566295c58a4b47d8b', 'f2e6e7d52a634e998856bc8523ac6f7a', '962a7985b0dd8efbdc5e22c72e6c7424', '380a25aef88db5b36a3cbab57eb60808', '53f4f3b2caa7652ddb669392b7

In [17]:
dataset_dir = '../data'
image_folder = os.path.join(dataset_dir, 'images')
labels_folder = os.path.join(dataset_dir, 'labels')

subfolders = ['train', 'val']
image_dirs = {subfolder: os.path.join(image_folder, subfolder) for subfolder in subfolders}
label_dirs = {subfolder: os.path.join(labels_folder, subfolder) for subfolder in subfolders}


for dir in image_dirs.values():
    os.makedirs(dir, exist_ok=True)
for dir in label_dirs.values():
    os.makedirs(dir, exist_ok=True)

image_files = [file for file in os.listdir(image_folder) if file.lower().endswith('.png')]

# split the files into training and validation sets
train_files, val_files = train_test_split(image_files, test_size=0.08, random_state=42)

# function to move files
def move_files(files, source_dir_images, source_dir_labels, target_dir_images, target_dir_labels):
    for file in files:
        shutil.move(os.path.join(source_dir_images, file), os.path.join(target_dir_images, file))
        # corresponding annotation file
        base_file_name = os.path.splitext(file)[0]
        annotation_file = base_file_name + '.txt'
        source_annotation_file = os.path.join(source_dir_labels, annotation_file)
        if os.path.exists(source_annotation_file):
            shutil.move(source_annotation_file, os.path.join(target_dir_labels, annotation_file))

move_files(train_files, image_folder, labels_folder, image_dirs['train'], label_dirs['train'])
move_files(val_files, image_folder, labels_folder, image_dirs['val'], label_dirs['val'])

print(f"Training set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")


Training set: 22894 images
Validation set: 1991 images


In [ ]:
# check the conversion

def draw_boxes_on_images(label_dir, image_dir, output_dir):
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        label_file_path = os.path.join(label_dir, label_file)
        
        base_filename = os.path.splitext(label_file)[0]
        image_file_path = os.path.join(image_dir, f"{base_filename}.png")
        if not os.path.exists(image_file_path):
            continue  
        
        image = cv2.imread(image_file_path)
        im_height, im_width = image.shape[:2]
        
        # bounding boxes from label 
        with open(label_file_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                parts = line.strip().split()
                class_id, x_center, y_center, width, height = map(float, parts)
                x_center, width = x_center * im_width, width * im_width
                y_center, height = y_center * im_height, height * im_height
                
                # convert from center coordinates to top left coordinates
                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)
                
                # draw rectangle on the image
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # save
        output_image_path = os.path.join(output_dir, f"{base_filename}.png")
        cv2.imwrite(output_image_path, image)
    
    print(f"Processed and saved all images with bounding boxes in {output_dir}")


label_directory = '..data/labels/val'
image_directory = '..data/images/val'
output_directory = '..data/images/val_bounded'

draw_boxes_on_images(label_directory, image_directory, output_directory)


In [9]:
def check_and_move_images(folder_path, corrupt_image_path, label_path, corrupt_label_path):
    corrupt_images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"): 
            image_path = os.path.join(folder_path, filename)
            # try to open
            img = cv2.imread(image_path)
            if img is None:
                corrupt_images.append(filename)
                # move corrupt image
                shutil.move(image_path, os.path.join(corrupt_image_path, filename))
                # find and move corresponding label file if exists
                label_file = filename[:-4] + '.txt'
                label_file_path = os.path.join(label_path, label_file)
                if os.path.exists(label_file_path):
                    shutil.move(label_file_path, os.path.join(corrupt_label_path, label_file))

    return corrupt_images


folder_path = '../data/images/train/'
corrupt_image_path = '../data/corrupt/images/train'
label_path = '../data/label/train'
corrupt_label_path = '../data/corrupt/label/train'

corrupt_images = check_and_move_images(folder_path, corrupt_image_path, label_path, corrupt_label_path)
print("done")


done


In [ ]:
# All the images are 1280 x 1204. We need to resize 1024 x 1024 with respect to the aspect ratio for annotations. So black padding will be applied.

def resize_and_pad(img, desired_size):
    old_size = img.shape[:2]  # old_size is in height, width format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # resize the image to new_size maintaining aspect ratio
    img = cv2.resize(img, (new_size[1], new_size[0]))

    # compute padding 
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    # add black padding to the image
    color = [0, 0, 0]  
    new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return new_img

def process_images_from_directory(input_dir, output_dir, desired_size):
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.png'):
            file_path = os.path.join(input_dir, filename)
            image = cv2.imread(file_path)
            if image is not None:
                processed_image = resize_and_pad(image, desired_size)
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, processed_image)
                print(f"saved: {output_path}")
            else:
                print(f"Failed to load image: {file_path}")

input_directory = '../data/images/train/'
output_directory = '../data/images/train1024/'
image_size = 1024 

process_images_from_directory(input_directory, output_directory, image_size)


In [2]:
# checking

from PIL import Image

image_path = '../data/images/train/0a0ba7d4c31cd8c12e66f3e792a9599f.png'
image = Image.open(image_path)
print(image.size)  


(1024, 1024)


In [ ]:
def adjust_labels(input_dir, output_dir, original_size, target_size):
    os.makedirs(output_dir, exist_ok=True)
    width, height = original_size
    scale = target_size / max(width, height)
    new_width = int(width * scale)
    new_height = int(height * scale)

    # padding added to each dimension
    pad_x = (target_size - new_width) / 2
    pad_y = (target_size - new_height) / 2

    for filename in os.listdir(input_dir):
        if filename.endswith('.txt'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            with open(input_path, 'r') as file, open(output_path, 'w') as outfile:
                for line in file:
                    parts = line.strip().split()
                    class_id = parts[0]
                    x_center = float(parts[1]) * width
                    y_center = float(parts[2]) * height
                    bbox_width = float(parts[3]) * width
                    bbox_height = float(parts[4]) * height

                    # adjust for the new dimensions
                    x_center = (x_center * scale + pad_x) / target_size
                    y_center = (y_center * scale + pad_y) / target_size
                    bbox_width = (bbox_width * scale) / target_size
                    bbox_height = (bbox_height * scale) / target_size

                    outfile.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

            print(f"Adjusted labels saved to {output_path}")


input_directory = '../data/labels/val_org'
output_directory = '../data/labels/val'
original_label_size = (1280, 1024)  
desired_label_size = 1024  

adjust_labels(input_directory, output_directory, original_label_size, desired_label_size)


In [7]:
def count_files(folder_path):
    files = os.listdir(folder_path)
    file_count = len([file for file in files if os.path.isfile(os.path.join(folder_path, file))])
    return file_count

directory_org = '..n/data/labels/val_org'
directory = '..n/data/labels/val'

print("Number of files in the label:", count_files(directory_org))
print("Number of files in the image:", count_files(directory))


Number of files in the label: 2323
Number of files in the image: 2323


In [1]:
def check_images(folder_path):
    total_files = 0
    failed_files = []

    for img_filename in os.listdir(folder_path):
        if img_filename.lower().endswith(('.png')):
            total_files += 1
            img_path = os.path.join(folder_path, img_filename)
            try:
                with Image.open(img_path) as img:
                    img.verify()  # check if image is broken
            except (IOError, SyntaxError) as e:
                failed_files.append(img_filename)
                print(f"Failed to open {img_path}: {e}")

    print(f"Total files checked: {total_files}")
    print(f"Failed files: {failed_files}")

train_dir = '..n/data/images/train'
val_dir = '..n/data/images/val'

print("Checking training images...")
check_images(train_dir)

print("Checking validation images...")
check_images(val_dir)


Checking training images...
Total files checked: 23478
Failed files: []
Checking validation images...
Total files checked: 2323
Failed files: []
